In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r"D:\aegis\Advance Python\Heath Care\diabetes.csv")
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
Pregnancies                 768 non-null int64
Glucose                     768 non-null int64
BloodPressure               768 non-null int64
SkinThickness               768 non-null int64
Insulin                     768 non-null int64
BMI                         768 non-null float64
DiabetesPedigreeFunction    768 non-null float64
Age                         768 non-null int64
Outcome                     768 non-null int64
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [4]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X=df.drop('Outcome',axis=1)
y=df.Outcome

In [7]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=20,stratify = y)

In [8]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
model_1 = RandomForestClassifier()
model_1.fit(X_train, y_train)
y_pred1 = model_1.predict(X_test)

In [10]:
from sklearn.metrics import accuracy_score,mean_squared_error

In [11]:
accuracy_score(y_test,y_pred1)

0.7792207792207793

In [12]:
from sklearn.model_selection import RandomizedSearchCV

In [13]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 110, num = 11)]
#max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

In [14]:
#Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [15]:
# Use the random grid to search for best hyperparameters

# First create the base model to tune
rf = RandomForestClassifier()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = param_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state = 10, n_jobs = -1)

# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   17.0s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  4.2min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [1, 11, 22, 33, 44, 55, 66,
                                                      77, 88, 99, 110],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=10, verbose=2)

In [16]:
rf_random.best_params_

{'n_estimators': 600,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 99,
 'bootstrap': False}

In [51]:
# Manually provide the best parameters to model for training
rf_reg = RandomForestClassifier(**{'n_estimators': 600,
 'min_samples_split':102,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 99,
 'bootstrap': False})

In [52]:
rf_reg.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, max_depth=99, max_features='sqrt',
                       min_samples_split=102, n_estimators=600)

In [53]:
y_pred2 = rf_reg.predict(X_test)

In [54]:
accuracy_score(y_test,y_pred2)

0.7857142857142857

In [55]:
print("Score based on testing data:")
print("Adjusted R-squared : {}%".format((rf_reg.score(X_test,y_test)*100).round(2)))
print("RMSE :", np.sqrt(mean_squared_error(y_pred2,y_test)).round(2))

Score based on testing data:
Adjusted R-squared : 78.57%
RMSE : 0.46


## Model 2:

In [23]:
from sklearn.preprocessing import StandardScaler

In [24]:
X=df.drop('Outcome',axis=1)
y=df.Outcome

In [25]:
sc = StandardScaler()
X_sc = sc.fit_transform(X)

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X_train,X_test,Y_train,Y_test= train_test_split(X_sc,y,test_size=0.2,random_state=20,stratify = y)

In [28]:
from sklearn.linear_model import LogisticRegression

In [29]:
lr= LogisticRegression()

In [30]:
lr.fit(X_train,Y_train)

LogisticRegression()

In [31]:
y_pred3 = lr.predict(X_test)

In [32]:
from sklearn.metrics import confusion_matrix,accuracy_score#, roc_curve, roc_auc_score, auc

In [33]:
print(accuracy_score(Y_test,y_pred3))

0.8116883116883117


In [34]:
np.mean(Y_test != y_pred3)

0.18831168831168832

## Model 3: Knn classifier

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X=df.drop('Outcome',axis=1)
y=df.Outcome

In [37]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=20,stratify = y)

In [38]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [39]:
#estimators = [('standardize', StandardScaler()),('knn', KNeighborsClassifier())] 
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('knn', KNeighborsClassifier()))
model = Pipeline(estimators)

In [40]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

kfold = StratifiedKFold(n_splits=10, shuffle = True, random_state=7)
results = cross_val_score(model, X_train, y_train, cv=kfold)

In [41]:
print(results.mean())

0.7135642517186673


In [42]:
print(results.std())

0.05167150214622143


In [43]:
search_space = [{
                 'knn__n_neighbors': range(1,20),
                 'knn__weights': ['distance']
                }]

In [44]:
from sklearn.model_selection import GridSearchCV

kfold = StratifiedKFold(n_splits=10, shuffle = True, random_state=10)
clf = GridSearchCV(model, search_space, cv=kfold, return_train_score=True ,verbose=False)
clf = clf.fit(X_train, y_train)

In [45]:
clf.best_estimator_

Pipeline(steps=[('standardize', StandardScaler()),
                ('knn',
                 KNeighborsClassifier(n_neighbors=19, weights='distance'))])

In [46]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('knn', KNeighborsClassifier(n_neighbors = 19)))
model = Pipeline(estimators)

In [47]:
model.fit(X_train, y_train)

Pipeline(steps=[('standardize', StandardScaler()),
                ('knn', KNeighborsClassifier(n_neighbors=19))])

In [57]:
Predicted_Class = model.predict(X_test)
print(accuracy_score(y_test,Predicted_Class))

0.8376623376623377


In [49]:
print("Score based on testing data:")
print("Adjusted R-squared : {}%".format((model.score(X_test,y_test)*100).round(2)))
print("RMSE :", np.sqrt(mean_squared_error(Predicted_Class,y_test)).round(2))

Score based on testing data:
Adjusted R-squared : 83.77%
RMSE : 0.4


In [56]:
import pickle
filename = 'Diabetes_model.pkl'
pickle.dump(model, open(filename, 'wb'))